In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class V2:
    def __init__(self, input_shape=(224, 224, 3), num_classes=1000):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build_model(self):
        inputs = Input(shape=self.input_shape)
        x = Conv2D(32, 3, strides=(2, 2), padding='same', use_bias=False)(inputs)
        x = BatchNormalization()(x)
        x = ReLU(6)(x)

        x = self.depth(x, stride=1, block_id=1)
        x = self.proj(x, num=16, block_id=1)
        x = self.bottleneck(x, t=6, num=24, stride=2, block_id=2)
        x = self.bottleneck(x, t=6, num=24, stride=1, block_id=3)
        x = self.bottleneck(x, t=6, num=32, stride=2, block_id=4)
        x = self.bottleneck(x, t=6, num=32, stride=1, block_id=5)
        x = self.bottleneck(x, t=6, num=32, stride=1, block_id=6)
        x = self.bottleneck(x, t=6, num=64, stride=2, block_id=7)
        x = self.bottleneck(x, t=6, num=64, stride=1, block_id=8)
        x = self.bottleneck(x, t=6, num=64, stride=1, block_id=9)
        x = self.bottleneck(x, t=6, num=64, stride=1, block_id=10)
        x = self.bottleneck(x, t=6, num=96, stride=1, block_id=11)
        x = self.bottleneck(x, t=6, num=96, stride=1, block_id=12)
        x = self.bottleneck(x, t=6, num=96, stride=1, block_id=13)
        x = self.bottleneck(x, t=6, num=160, stride=2, block_id=14)
        x = self.bottleneck(x, t=6, num=160, stride=1, block_id=15)
        x = self.bottleneck(x, t=6, num=160, stride=1, block_id=16)
        x = self.bottleneck(x, t=6, num=320, stride=1, block_id=17)

        x = Conv2D(filters=1280, kernel_size=1, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU(6)(x)

        x = GlobalAveragePooling2D()(x)
        output = Dense(self.num_classes, activation='softmax')(x)

        model = tf.keras.Model(inputs, output)
        return model

    def depth(self, inputs, kernel_size=3, strides=(1, 1), padding='same', activation='relu'):
        x = DepthwiseConv2D(kernel_size, strides=strides, padding=padding, activation=activation)(inputs)
        x = BatchNormalization()(x)
        return x

    def proj(self, inputs, num, kernel_size=1, strides=(1, 1), padding='same', activation='relu'):
        x = Conv2D(num, kernel_size, strides=strides, padding=padding, activation=activation, use_bias=False)(inputs)
        x = BatchNormalization()(x)
        return x

    def bottleneck(self, inputs, t, num, kernel_size=3, strides=(1, 1), padding='same', activation='relu', block_id=None):
        expand = t * num
        x = Conv2D(expand, 1, padding='same', activation=activation, use_bias=False)(inputs)
        x = BatchNormalization()(x)
        x = self.depth(x, kernel_size, strides, padding, activation)
        x = self.proj(x, num, activation=activation)
        return x

    def taug(self):
        train_datagen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')
        return train_datagen

    def vaug(self):
        validation_datagen = ImageDataGenerator()
        return validation_datagen